# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [9]:
#Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [====================

Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [10]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [11]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [12]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - ETA: 0s - loss: nan - mse: na - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/3

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4998 - mse: 0.4998 - val_loss: 0.1821 - val_mse: 0.1821
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2717 - mse: 0.2717 - val_loss: 0.1345 - val_mse: 0.1345
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2178 - mse: 0.2178 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1900 - mse: 0.1900 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1659 - mse: 0.1659 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1539 - mse: 0.1539 - val_loss: 0.1014 - val_mse: 0.1014
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1430 - mse: 0.1430 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1352 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1357 - val_mse: 0.1357
Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1377 - val_mse: 0.1377
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0218 - mse: 0

Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1555 - val_mse: 0.1555
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1535 - val_mse: 0.1535
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1524 - val_mse: 0.1524
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1539 - val_mse: 0.1539
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1542 - val_mse: 0.1542
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 911us/step - loss: 0.0078 - mse: 0.0078


[0.007770191878080368, 0.007770191878080368]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 889us/step - loss: 0.1588 - mse: 0.1588


[0.15883757174015045, 0.15883757174015045]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [17]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

31319.017431312168

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [18]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer


# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))


Epoch 1/150

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

33/33 [==============================] - 0s 6ms/step - loss: 0.5471 - mse: 0.5471 - val_loss: 0.2670 - val_mse: 0.2670
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2522 - mse: 0.2522 - val_loss: 0.1944 - val_mse: 0.1944
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2031 - mse: 0.2031 - val_loss: 0.1704 - val_mse: 0.1704
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1795 - mse: 0.1795 - val_loss: 0.1557 - 

Epoch 58/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0397 - mse: 0.0397 - val_loss: 0.1632 - val_mse: 0.1632
Epoch 59/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0397 - mse: 0.0397 - val_loss: 0.1618 - val_mse: 0.1618
Epoch 60/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0389 - mse: 0.0389 - val_loss: 0.1603 - val_mse: 0.1603
Epoch 61/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0386 - mse: 0.0386 - val_loss: 0.1614 - val_mse: 0.1614
Epoch 62/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0382 - mse: 0.0382 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0375 - mse: 0.0375 - val_loss: 0.1602 - val_mse: 0.1602
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0372 - mse: 0.0372 - val_loss: 0.1612 - val_mse: 0.1612
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

33/33 [==============================] - 0s 1ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1670 - val_mse: 0.1670
Epoch 120/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 121/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1707 - val_mse: 0.1707
Epoch 122/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 123/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1651 - val_mse: 0.1651
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1640 - val_mse: 0.1640
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1670 - val_mse: 0.1670
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0205 -

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [21]:
# Evaluate the model on training data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 889us/step - loss: 0.1662 - mse: 0.1662


[0.16617237031459808, 0.16617237031459808]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [22]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4412 - mse: 0.4412 - val_loss: 0.1848 - val_mse: 0.1848
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2497 - mse: 0.2497 - val_loss: 0.1819 - val_mse: 0.1819
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1984 - mse: 0.1984 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1737 - mse: 0.1737 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1513 - mse: 0.1513 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1360 - mse: 0.1360 - val_loss: 0.1180 - val_mse: 0.1180
Epoch 7/150
33/33 [==============================] - 0s 978us/step - loss: 0.1280 - mse: 0.1280 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1156 -

33/33 [==============================] - 0s 909us/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 63/150
33/33 [==============================] - 0s 939us/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 64/150
33/33 [==============================] - 0s 848us/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 65/150
33/33 [==============================] - 0s 849us/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 66/150
33/33 [==============================] - 0s 855us/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 67/150
33/33 [==============================] - 0s 833us/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 68/150
33/33 [==============================] - 0s 788us/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 69/150
33/33 [==============================] - 0s 856us/step - loss:

Epoch 123/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1346 - val_mse: 0.1346
Epoch 124/150
33/33 [==============================] - 0s 879us/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 125/150
33/33 [==============================] - 0s 933us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 126/150
33/33 [==============================] - 0s 788us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 127/150
33/33 [==============================] - 0s 793us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1337 - val_mse: 0.1337
Epoch 128/150
33/33 [==============================] - 0s 758us/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 129/150
33/33 [==============================] - 0s 818us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1341 - val_mse: 0.1341
Epoch 130/150
33/33 [==============================] - 0s

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 958us/step - loss: 0.0085 - mse: 0.0085


[0.008493739180266857, 0.008493739180266857]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 894us/step - loss: 0.1363 - mse: 0.1363


[0.13632521033287048, 0.13632521033287048]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [26]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4162 - mse: 0.4162 - val_loss: 0.1285 - val_mse: 0.1285
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1763 - mse: 0.1763 - val_loss: 0.1730 - val_mse: 0.1730
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1340 - mse: 0.1340 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1058 - mse: 0.1058 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0990 - mse: 0.0990 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 6/150
33/33 [==============================] - 0s 1000us/step - loss: 0.0790 - mse: 0.0790 - val_loss: 0.0902 - val_mse: 0.0902
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0670 - mse: 0.0670 - val_loss: 0.1057 - val_mse: 0.1057
Epoch 8/150
33/33 [==============================] - 0s 970us/step - loss: 0.057

Epoch 63/150
33/33 [==============================] - 0s 914us/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 64/150
33/33 [==============================] - 0s 970us/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1240 - val_mse: 0.1240
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.0896 - val_mse: 0.0896
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0892 - val_mse: 0.0892
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.0903 - val_mse: 0.0903
Epoch 69/150
33/33 [==============================] - 0s 909us/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 70/150
33/33 [==============================] - 0s 879us/step - 

33/33 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0862 - val_mse: 0.0862
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0808 - val_mse: 0.0808
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0969 - val_mse: 0.0969
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0975 - val_mse: 0.0975
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0856 - val_mse: 0.0856
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0832 - val_mse: 0.0832
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0931 - val_mse: 0.0931
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0050 -

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 977us/step - loss: 0.0022 - mse: 0.0022


[0.002166061196476221, 0.002166061196476221]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 778us/step - loss: 0.0818 - mse: 0.0818


[0.08184388279914856, 0.08184388279914856]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [29]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])


# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: 0.5299 - mse: 0.5299 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0.1907 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1380 - mse: 0.1380 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1137 - mse: 0.1137 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0830 - mse: 0.0830 - val_loss: 0.1292 - val_mse: 0.1292
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0645 - mse: 0.0645 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0477 - mse: 0.0477 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0373 - m

33/33 [==============================] - 0s 1000us/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1133 - val_mse: 0.1133
Epoch 63/150
33/33 [==============================] - 0s 909us/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1256 - val_mse: 0.1256
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1350 - val_mse: 0.1350
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0054 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0975 - val_mse: 0.0975
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0926 - val_mse: 0.0926
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.0918 - val_mse: 0.0918
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0015 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 909us/step - loss: 0.0035 - mse: 0.0035


[0.0035423680674284697, 0.0035423680674284697]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [31]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 1ms/step - loss: 0.0860 - mse: 0.0860


[0.08604924380779266, 0.08604924380779266]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [32]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)


5/5 [==============================] - 0s 1ms/step - loss: 0.1364 - mse: 0.1364


[0.1364329308271408, 0.1364329308271408]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [33]:

# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29026.260914785118

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.